# Baseline Text Classification


ACEInhibitors dataset from: https://www.dropbox.com/sh/ud5sf1fy6m7o219/AAD9pkY5gYe_XYV2oHDw68uva?dl=0

## Libraries and methods

In [ ]:
import warnings
warnings.filterwarnings("ignore")

!pip install -Uqq ipdb
import ipdb

     |████████████████████████████████| 793 kB 8.3 MB/s 
     |████████████████████████████████| 381 kB 52.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.34.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
path = "/content/gdrive/MyDrive/Magistrale/Stage/data"
root_dir = "/content/gdrive/MyDrive"

In [ ]:
#libraries
import csv
import numpy as np
import pandas as pd
import math
import re
import nltk
nltk.download('punkt')
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import regexp_tokenize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, fbeta_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from pathlib import Path  

In [ ]:
#preprocessing methods 

def listToString(s): 
    str1 = "" 
    for ele in s: 
        str1 = str1 + " " + ele
    return str1

def clean_text(df, col):
    """A function for keeping only alpha-numeric characters and replacing all 
    white space with a single space.
    """

    #[^A-Za-z0-9]+: regex to match a string of characters that are not a letters or numbers
    return df[col].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', str(x).lower()))\
                  .apply(lambda x: re.sub('\s+', ' ', x).strip())

def flatten_words(list1d, get_unique=False):
    qa = [s.split() for s in list1d]
    if get_unique:
        return sorted(list(set([w for sent in qa for w in sent])))
    else:
        return [w for sent in qa for w in sent]

def count_words(df, column):
    df = df.copy()

    # df['n_questionmarks'] = count_pattern(df, text, '\?')
    # df['n_periods'] = count_pattern(df, text, '\.')
    # df['n_apostrophes'] = count_pattern(df, text, '\'')
    # df['first_word'] = df[text_clean].apply(lambda x: split_on_word(x)[0])
    # question_words = ['what', 'how', 'why', 'is']
    # for w in question_words:
    #     col_wc = 'n_' + w
    #     col_fw = 'fw_' + w
    #     df[col_wc] = count_pattern(df, text_clean, w)
    #     df[col_fw] = (df.first_word == w) * 1
        
    # del df['first_word']
    
    col_name = "n_words_in_" + column
    df[col_name] = df[column].apply(lambda x: len(split_on_word(x)))

    return df

def count_pattern(df, col, pattern):
    """Count the occurrences of `pattern`
    in df[col].
    """
    df = df.copy()
    return df[col].str.count(pattern)

def split_on_word(text):
    """Use regular expression tokenizer.
    Keep apostrophes.
    Returns a list of lists, one list for each sentence:
        [[word, word], [word, word, ..., word], ...].
    """
    if type(text) is list:
        return [regexp_tokenize(sentence, pattern="\w+(?:[-']\w+)*") for sentence in text]
    else:
        return regexp_tokenize(text, pattern="\w+(?:[-']\w+)*")

In [ ]:
def from_list_values_to_columns(col, k, print=False): 

#obtaining the unique values 

  dataset[k][col] = dataset[k][col].apply(eval)

  col_dict = {} 
  for i in dataset[k][col]: #obtain value_count in a dictionary
    for j in i:
        if j not in col_dict:
            col_dict[j] = 1 #new column
        else:
            col_dict[j] += 1 #update column count

  series = pd.Series([x for _list in dataset[k][col] for x in _list]) #reducing its dimensions from 2 to 1 

  if print == True:
    print(series.value_counts()) #display value count

#creating new binary columns 

  bool_dict = {} #create boolean dict (the binary value for every colum in col_dict and for every row in the dataset[k])
  for i, item in enumerate(col_dict.keys()): 
    bool_dict[item] = dataset[k][col].apply(lambda x: item in x)

  return pd.DataFrame(bool_dict).astype(int)

In [ ]:
def update_col_names(feature, df, sub_features): 

  #useful trasnformation for assigning a list to a dataframe cell
  l = col_names.index[col_names["feature"] == feature].tolist()
  col_names.at[l[0], df] = sub_features

In [ ]:
def find_documents_about_topic(df, column, new_column, l): 

  x = df[column][df[column].str.contains('|'.join(l))] #rows in df[column] that contains at least 1 item of "words"
  
  df[new_column] = 0
  df.loc[df.index.isin(x.index), new_column] = 1 #assigning 1 to the corresponding rows of x in df

  print("Number of documents that", new_column, ":", len(x))

  return df

In [ ]:
def evalmetrics(y_test, test_predicted, labels, new_row, res): 

  #accuracy
  accuracy = accuracy_score(y_test, test_predicted)

  #precision
  precision = precision_score(y_test, test_predicted, average='binary', zero_division=0)
  print("\nPrecision with dataset", new_row["df"], ":", precision, "\n")

  #recall
  recall = recall_score(y_test, test_predicted, average='binary', zero_division=0)
  print("Recall with dataset", new_row["df"], ":", recall, "\n")

  #f-2 score
  f2 = fbeta_score(y_test, test_predicted, average='binary', beta=2)
  print("F-2 score with dataset", new_row["df"], ":", f2, "\n")

  #f-3 score
  f3 = fbeta_score(y_test, test_predicted, average='binary', beta=3)
  print("F-3 score with dataset", new_row["df"], ":", f3, "\n")

  #confusion matrix
  cm = confusion_matrix(y_test, test_predicted, labels=labels)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm)
  disp.plot()
  title = new_row["model"] + " confusion matrix on dataset " + new_row["df"]
  disp.ax_.set_title(title)

  #specificity or true negative rate
  FP = cm.sum(axis=0) - np.diag(cm)  
  FN = cm.sum(axis=1) - np.diag(cm)
  TP = np.diag(cm)
  TN = cm.sum() - (FP + FN + TP)
  TNR = TN/(TN+FP)
  print("True negative rate with dataset", new_row["df"], ":", TNR[1], "\n")

  #evaluation metrics
  report = classification_report(y_test, test_predicted, output_dict=True)
  display(pd.DataFrame(report).transpose().round(decimals=3))

  new_row.update({'accuracy' : accuracy, 'precision' : precision, 'recall' : recall, 'true_negative_rate' : TNR[1], 'f2_score' : f2, 'f3_score' : f3})
  res = res.append(new_row, ignore_index=True)

  return res

In [ ]:
def compute_avg_std(res): 
  
  avg_res = pd.DataFrame() 

  for k in res["df"].unique(): 
    for j in res["model"].unique(): 
      condition = (res["df"] == k) & (res["model"] == j)

      new_row = {"df": k, "model":j}
      avg_res = avg_res.append(new_row, ignore_index=True)
      new_condition = (avg_res["df"] == k) & (avg_res["model"] == j)

      for column in res.columns[5:]: #every column from precision column

        avg_column = column + "_avg"
        std_column = column + "_std"
        
        avg_res.loc[new_condition, avg_column] = round(res.loc[condition, column].mean(), 2)
        avg_res.loc[new_condition, std_column] = round(res.loc[condition, column].std(), 2)

  return avg_res

## Import of data

In [ ]:
dataset = dict()

### ACE Inhibitors dataset

In [ ]:
#import of data
from_drive = True

if from_drive == True: 
  dataset["ace"] = pd.read_csv(path + "/ACEInhibitors.tsv", sep='\t')
else: 
  dataset["ace"] = pd.read_csv('/content/ACEInhibitors.tsv', sep='\t')

In [ ]:
print(dataset["ace"].shape)
pd.set_option('display.max_columns', None)
display(dataset["ace"].head(3))

#### Columns distribution and selection

In [ ]:
def print_value_counts(l, df): 
  for item in l: 
    print(dataset[df][item].value_counts(), "\n")

def rename_columns(names, df): 
  for key in names: 
    dataset[df].rename(columns={key : names[key]}, inplace=True)

In [ ]:
#columns 
print(dataset["ace"].columns)

In [ ]:
#value counts
variables = ["MH", "STAT", "VI", "IP", "DP", "FAU", "AU", "AD", "LA", "PT", "PL", "TA", "JT"] 
print_value_counts(l=variables, df="ace")

In [ ]:
#target distribution
dataset["ace"]["Label"].value_counts()

In [ ]:
#renaming columns
names = {"DP" : "publication_date", "FAU" : "full_authors", "PT" : "publication_type", "PL" : "publication_place", "TA" : "journal_title_abbreviation", "MH" : "mesh_terms"}
rename_columns(names=names, df="ace")

#column selection
dataset["ace"] = dataset["ace"][["publication_date", "full_authors", "publication_type", "publication_place", "journal_title_abbreviation", "Title", "Abstract", "mesh_terms", "Label"]]

### Chronic obstructive pulmonary disease (COPD) dataset

In [ ]:
#import of copd dataset
from_drive = True

if from_drive == True: 
  dataset["copd"] = pd.read_csv(path + "/copd.tsv", sep='\t')
else: 
  dataset["copd"] = pd.read_csv('/content/copd.tsv', sep='\t')

In [ ]:
print(dataset["copd"].shape)
display(dataset["copd"].head(3))

In [ ]:
#check if Title is identical to Abstract
print(all(dataset["copd"]["Title"] == dataset["copd"]["Abstract"]))
dataset["copd"].drop('Title', inplace=True, axis=1) #drop Title

In [ ]:
dataset["copd"]["Label"].value_counts()

### Proton Pump Inhibitors dataset

In [ ]:
#import of ppi dataset
from_drive = True

if from_drive == True: 
  dataset["ppi"] = pd.read_csv(path + "/ProtonPumpInhibitors.tsv", sep='\t')
else: 
  dataset["ppi"] = pd.read_csv('/content/ProtonPumpInhibitors.tsv', sep='\t')

In [ ]:
print(dataset["ppi"].shape)
display(dataset["ppi"].head(3))

#### Columns distribution and selection

In [ ]:
#columns
print(dataset["ppi"].columns)

In [ ]:
#value counts
print_value_counts(l=variables, df="ppi")

In [ ]:
#target distribution
dataset["ppi"]["Label"].value_counts()

In [ ]:
names

In [ ]:
#renaming columns
rename_columns(names=names, df="ppi")

#column selection
dataset["ppi"] = dataset["ppi"][["publication_date", "full_authors", "publication_type", "publication_place", "journal_title_abbreviation", "Title", "Abstract", "mesh_terms", "Label"]]

In [ ]:
dataset["ppi"]

## Columns preprocessing

### Text preprocessig

- Checking missing values, 
- Concatenating `Title`, `Abstract` and `mesh_terms` in `text`,
- Preprocessing the text in `text_clean`,
- Removing missing values.

In [ ]:
## Variable mesh_terms

for i in dataset: 

  if i != "copd": 

    #replace missing values with empty list
    dataset[i]["mesh_terms"] = dataset[i]["mesh_terms"].replace(np.NaN, "[]")

    #from list of strings to string
    for index, value in enumerate(dataset[i]["mesh_terms"]): 
      dataset[i].loc[index, "mesh_terms"] = listToString(eval(value))
      
    dataset[i]['mesh_terms'] = dataset[i]['mesh_terms'].str.replace('[/*]',' ')
    dataset[i] = count_words(dataset[i], "mesh_terms")

In [ ]:
## Variables Title ad Abstract

for i in dataset: 

  #count number of words in Title
  if i != "copd":
    dataset[i] = count_words(dataset[i], "Title")

  #count number of words in Abstract
  dataset[i]["Abstract"] = dataset[i]["Abstract"].replace(np.NaN, "[]") #replace missing values with empty list
  dataset[i]["Abstract"] = dataset[i]["Abstract"].astype(str) 
  dataset[i] = count_words(dataset[i], "Abstract")

In [ ]:
for i in dataset: 

  #checking missing values
  print("Dataset", i, "shape:", dataset[i].shape)
  print(dataset[i].isnull().sum(axis=0), "\n")

  #concatenating Title, Abstract (and mesh_terms)
  if i != "copd": 
    dataset[i]['text'] = dataset[i]['Title'] + (dataset[i]['Abstract']).fillna(' ') + (dataset[i]['mesh_terms']).fillna(' ')
    dataset[i].drop(['Title', "Abstract", "mesh_terms"], inplace=True, axis=1)
  else:
    dataset[i]['text'] = (dataset[i]['Abstract']).fillna(' ')
    dataset[i].drop("Abstract", inplace=True, axis=1)
    
  ## Variable text_clean

  #text preprocessing
  dataset[i]['text_clean'] = clean_text(dataset[i], 'text')
  dataset[i] = count_words(dataset[i], "text_clean")

  #removing Nan and checking missing values again
  #print("Missing values after creating \"text\":")
  #print(dataset[i].isnull().sum(axis=0))
  dataset[i] = dataset[i].dropna()
  dataset[i] = dataset[i].reset_index()
  print("New dataset shape:", dataset[i].shape, "\n")


### Features preprocessing

In [ ]:
#creating col_names (used to store columns' names)
col_names = pd.DataFrame(columns = ["feature", "ace", "ppi"])
col_names = col_names.append({"feature": "publication_type"}, ignore_index = True)
col_names = col_names.append({"feature": "full_authors"}, ignore_index = True)

In [ ]:
for i in dataset: 

  if i != "copd": 

    ## Variable publication_date

    #removing the day and maintaining only year and month
    for index, content in enumerate(dataset[i]["publication_date"]): 
      if len(nltk.word_tokenize(content)) > 2:
        dataset[i].loc[index, "publication_date"] = " ".join(nltk.word_tokenize(content)[0:2])
    #print(dataset[i]["publication_date"].value_counts(), "\n")
    

    ## Variable publication_type

    temp = from_list_values_to_columns("publication_type", i)
    update_col_names("publication_type", i, list(temp.columns))
    dataset[i].drop("publication_type", inplace=True, axis=1)
    dataset[i] = dataset[i].join(temp)
    

    ## Variable full_authors

    temp = from_list_values_to_columns("full_authors", i)
    update_col_names("full_authors", i, list(temp.columns))
    dataset[i].drop("full_authors", inplace=True, axis=1)
    dataset[i] = dataset[i].join(temp)

In [ ]:
col_names

## Topic search

### Ace inhibitors related documents

In [ ]:
list1 = ["ace", "angiotensin converting enzyme"]
list2 = ["alacepril", "captopril", "zofenopril", "enalapril", "ramipril", 
          "quinapril", "perindopril", "lisinopril", "benazepril", "imidapril", 
          "trandolapril", "cilazapril", "fosinopril", "moexipril"]

In [ ]:
dataset["ace"] = find_documents_about_topic(dataset["ace"], "text_clean", "contains_topic", list1)
dataset["ace"] = find_documents_about_topic(dataset["ace"], "text_clean", "contains_other_topic", list2)

In [ ]:
print("Checking how many documents don't contain any of the searched words:\n")
print(dataset["ace"][["contains_topic", "contains_other_topic"]].eq(0).all(1).value_counts(), "\n")
print("True -> they don't contain any of the words")
print("False -> they contain at list one word")

### COPD related documents

In [ ]:
list1 = ["chronic obstructive pulmonary disease", "copd"]
list2 = ["chronic obstructive lung disease", "cold", "chronic obstructive airway disease", "coad"]

In [ ]:
dataset["copd"] = find_documents_about_topic(dataset["copd"], "text_clean", "contains_topic", list1)
dataset["copd"] = find_documents_about_topic(dataset["copd"], "text_clean", "contains_other_topic", list2)

In [ ]:
print("Checking how many documents don't contain any of the searched words:\n")
print(dataset["copd"][["contains_topic", "contains_other_topic"]].eq(0).all(1).value_counts(), "\n")
print("True -> they don't contain any of the words")
print("False -> they contain at list one word")

### PPI related documents

In [ ]:
list1 = ["proton pump inhibitors", "ppi", "ppis"]
list2 = ["omeprazole", "lansoprazole", "dexlansoprazole", "esomeprazole", "pantoprazole", "rabeprazole", "ilaprazole"]

In [ ]:
dataset["ppi"] = find_documents_about_topic(dataset["ppi"], "text_clean", "contains_topic", list1)
dataset["ppi"] = find_documents_about_topic(dataset["ppi"], "text_clean", "contains_other_topic", list2)

In [ ]:
print("Checking how many documents don't contain any of the searched words:\n")
print(dataset["ppi"][["contains_topic", "contains_other_topic"]].eq(0).all(1).value_counts(), "\n")
print("True -> they don't contain any of the words")
print("False -> they contain at list one word")

In [ ]:
#updating col_names
col_names = col_names.append({"feature" : ["contains_topic"], "ace" : ["contains_topic"], "ppi" : ["contains_topic"], "copd" : ["contains_topic"]}, ignore_index=True) 
col_names = col_names.append({"feature" : ["contains_other_topic"], "ace" : ["contains_other_topic"], "ppi" : ["contains_other_topic"], "copd" : ["contains_other_topic"]}, ignore_index=True) 
col_names = col_names.append({"feature" : ["n_words_in_mesh_terms"], "ace" : ["n_words_in_mesh_terms"], "ppi" : ["n_words_in_mesh_terms"]}, ignore_index=True) 
col_names = col_names.append({"feature" : ["n_words_in_Title"], "ace" : ["n_words_in_Title"], "ppi" : ["n_words_in_Title"]}, ignore_index=True) 
col_names = col_names.append({"feature" : ["n_words_in_Abstract"], "ace" : ["n_words_in_Abstract"], "ppi" : ["n_words_in_Abstract"], "copd" : ["n_words_in_Abstract"]}, ignore_index=True) 
col_names = col_names.append({"feature" : ["n_words_in_text_clean"], "ace" : ["n_words_in_text_clean"], "ppi" : ["n_words_in_text_clean"], "copd" : ["n_words_in_text_clean"]}, ignore_index=True) 
col_names

### Final data preprocessing


#### One hot encode for categorical variables

In [ ]:
def update_col_names(feature, df, sub_features): 

  #useful trasnformation for assigning a list to a dataframe cell
  l = col_names.index[col_names["feature"] == feature].tolist()
  col_names.at[l[0], df] = sub_features

In [ ]:
#one hot encode for categorical variables

enc = OneHotEncoder()
cat_features = ['publication_date','publication_place','journal_title_abbreviation'] #categorical features

#create rows in col_names
col_names = col_names.append({"feature": "publication_date"}, ignore_index = True)
col_names = col_names.append({"feature": "publication_place"}, ignore_index = True)
col_names = col_names.append({"feature": "journal_title_abbreviation"}, ignore_index = True)

for i in dataset:

  if i != "copd": #doesn't have any categorical variable

    enc_df = pd.DataFrame(enc.fit_transform(dataset[i][cat_features]).toarray()) #one hot encode df for categorical features
    enc_df.columns = enc.get_feature_names(cat_features) #renaming columns of enc_df
    print("One hot encode dataset shape:", enc_df.shape)

    dataset[i] = dataset[i].join(enc_df)
    dataset[i] = dataset[i].drop(cat_features, axis=1)

    #updating col_names
    update_col_names(feature="publication_date", df=i, 
                     sub_features=[x for x in enc_df.columns if x.startswith("publication_date")])
    update_col_names(feature="publication_place", df=i, 
                     sub_features=[x for x in enc_df.columns if x.startswith("publication_place")])
    update_col_names(feature="journal_title_abbreviation", df=i, 
                     sub_features=[x for x in enc_df.columns if x.startswith("journal")])
    
  dataset[i] = dataset[i].drop("index", axis=1)
  print("Final", i, "shape:", dataset[i].shape, "\n")

del enc_df

In [ ]:
col_names

In [ ]:
#saving the preprocessed datasets and col_names in csv files (to use them in "Dataset statistics by class")
if from_drive == True: 
  with open(path + "/preprocessed_ace.csv", 'w', encoding = 'utf-8-sig') as f:
    dataset["ace"].to_csv(f) 
  with open(path + "/preprocessed_copd.csv", 'w', encoding = 'utf-8-sig') as f:
    dataset["copd"].to_csv(f) 
  with open(path + "/preprocessed_ppi.csv", 'w', encoding = 'utf-8-sig') as f:
    dataset["ppi"].to_csv(f) 
  with open(path + "/columns_names.csv", 'w', encoding = 'utf-8-sig') as f:
    col_names.to_csv(f) 
else: 
  filepath = Path('/content/preprocessed_acet.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)  
  dataset["ace"].to_csv(filepath)
  filepath = Path('/content/preprocessed_copd.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)  
  dataset["copd"].to_csv(filepath)
  filepath = Path('/content/preprocessed_ppi.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)
  dataset["ppi"].to_csv(filepath)
  filepath = Path('/content/columns_names.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)  
  col_names.to_csv(filepath)

### Feature importance and feature selection

In [ ]:
# creating the random forest that computes feature importance

for i in dataset:

  forest = RandomForestClassifier(random_state=0)
  forest.fit(dataset[i][dataset[i].columns.difference(["text", "text_clean", "Label"])], dataset[i]["Label"])

  #computing feature importance
  importances = forest.feature_importances_

  features = list(set(dataset[i].columns) - set(["text", "text_clean", "Label"])) #all columns except text, text_clean and Label

  forest_importances = pd.Series(importances, index=features) 
  forest_importances = forest_importances.sort_values(ascending=False)

  print("\nInitial number of features:", len(forest_importances))

  #non-important features
  non_important_features = forest_importances.where(forest_importances <= 0.002)
  non_important_features = non_important_features.dropna()

  #important features
  forest_importances = forest_importances.where(forest_importances > 0.002)
  forest_importances = forest_importances.dropna()
  print("Number of features after removing the ones with equal or less than 0.002 importance:", len(forest_importances), "\n")

  #visualization of the top 20 features sorted by importance
  fig, ax = plt.subplots()
  forest_importances.head(20).plot.bar(ax=ax)
  plt.title("Feature importances using MDI")
  ax.set_ylabel("Mean decrease in impurity")
  plt.show()

  #removing non important features from the dataset
  dataset[i].drop(non_important_features.index, axis = 1, inplace=True)

In [ ]:
#saving the preprocessed datasets and col_names in csv files (to use them in "Dataset statistics by class")
if from_drive == True: 
  with open(path + "/preprocessed_ace2.csv", 'w', encoding = 'utf-8-sig') as f:
    dataset["ace"].to_csv(f) 
  with open(path + "/preprocessed_copd2.csv", 'w', encoding = 'utf-8-sig') as f:
    dataset["copd"].to_csv(f) 
  with open(path + "/preprocessed_ppi2.csv", 'w', encoding = 'utf-8-sig') as f:
    dataset["ppi"].to_csv(f) 
else: 
  filepath = Path('/content/preprocessed_ace2.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)  
  dataset["ace"].to_csv(filepath)
  filepath = Path('/content/preprocessed_copd2.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)  
  dataset["copd"].to_csv(filepath)
  filepath = Path('/content/preprocessed_ppi2.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)
  dataset["ppi"].to_csv(filepath)

## Models training and resuls

In [ ]:
#creating dataset to save results
res = pd.DataFrame() 

#cross validation k=5
skf = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)

pred = pd.DataFrame()

for i in dataset: 

  print("\nDATASET", i, "\n")

  X = dataset[i].loc[:, dataset[i].columns != 'Label']
  y = dataset[i]["Label"]

  j = 0

  for train_index, test_index in skf.split(X, y):

    j = j + 1
    print("\nSplit number", j, "\n")
  
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #resetting indexes for concat()
    X_train = X_train.reset_index(drop=True) 
    X_test = X_test.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    
    #tf-idf
    all_text = X_train["text_clean"].values.tolist() + X_test["text_clean"].values.tolist()
    vocab = flatten_words(all_text, get_unique=True)
    tfidf = TfidfVectorizer(stop_words='english', vocabulary=vocab)
    training_matrix = tfidf.fit_transform(X_train["text_clean"])
    test_matrix = tfidf.fit_transform(X_test["text_clean"])

    print("Training matrix:", training_matrix.shape)
    print("Test matrix:", test_matrix.shape)

    X_train = pd.concat([X_train, pd.DataFrame(training_matrix.todense())], axis=1) #add training_matrix to X_train
    X_test = pd.concat([X_test, pd.DataFrame(test_matrix.todense())], axis=1)


    #scaling data
    scaler = StandardScaler()
    features = list(set(X_train.columns) - set(["text", "text_clean", "Label"])) #all columns except text, text_clean and Label
    X_train = scaler.fit_transform(X_train[features].values)
    y_train = y_train.values
    X_test = scaler.transform(X_test[features].values)

    #####

    ## SVM training

    svm = SVC(kernel = "linear", class_weight="balanced", random_state=1, probability=True)
    svm = svm.fit(X_train, y_train)

    ## SVM results

    #prediction on the test set
    test_predicted = svm.predict(X_test) #predicted classes
    test_preds = svm.predict_proba(X_test) #predicted probabilities
    test_preds = test_preds[:,1]
    new_row = {"df" : i, "model" : "SVM", "fold": j, "pred" : pd.DataFrame(test_preds), "target" : pd.DataFrame(y_test)}
    pred = pred.append(new_row, ignore_index=True)

    #evaluation metrics
    print("\nSVM RESULTS:\n")
    new_row = {"df" : i, "train_shape" : X_train.shape, "test_shape" : X_test.shape, 
               "fold" : j, "model" : "SVM"}
    res = evalmetrics(y_test, test_predicted, svm.classes_, new_row, res)

    ######

    ## DT training

    dt = tree.DecisionTreeClassifier(class_weight="balanced", random_state=1)
    dt = dt.fit(X_train, y_train)

    ## DT results

    #prediction on the test set
    test_predicted = dt.predict(X_test) #predicted classes
    test_preds = dt.predict_proba(X_test) #predicted probabilities
    test_preds = test_preds[:,1]
    new_row = {"df" : i, "model" : "DT", "fold": j, "pred" : pd.DataFrame(test_preds), "target" : pd.DataFrame(y_test)}
    pred = pred.append(new_row, ignore_index=True)

    #evaluation metrics
    print("\nDT RESULTS:\n")
    new_row = {"df" : i, "train_shape" : X_train.shape, "test_shape" : X_test.shape, 
               "fold" : j, "model" : "DT"}
    res = evalmetrics(y_test, test_predicted, svm.classes_, new_row, res)

    ######

    # RF training

    forest = RandomForestClassifier(class_weight="balanced", random_state=1)
    forest = forest.fit(X_train, y_train)
    ## RF results

    #prediction on the test set
    test_predicted = forest.predict(X_test) #predicted classes
    test_preds = forest.predict_proba(X_test) #predicted probabilities
    test_preds = test_preds[:,1]
    new_row = {"df" : i, "model" : "RF", "fold": j, "pred" : pd.DataFrame(test_preds), "target" : pd.DataFrame(y_test)}
    pred = pred.append(new_row, ignore_index=True)

    #evaluation metrics
    print("\nRF RESULTS:\n")
    new_row = {"df" : i, "train_shape" : X_train.shape, "test_shape" : X_test.shape, 
               "fold" : j, "model" : "RF"}
    res = evalmetrics(y_test, test_predicted, svm.classes_, new_row, res)

# computation of mean metrics
avg_res = compute_avg_std(res)

In [ ]:
res.shape

In [ ]:
res.head(10)

In [ ]:
avg_res.shape

In [ ]:
avg_res.head()

In [ ]:
#saving results in a csv file
if from_drive == True: 
  with open(path + "/ml_out.csv", 'w', encoding = 'utf-8-sig') as f:
    res.to_csv(f)  
  with open(path + "/ml_avg_out.csv", 'w', encoding = 'utf-8-sig') as f:
    avg_res.to_csv(f) 
else: 
  filepath = Path('/content/ml_out.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)  
  res.to_csv(filepath)
  filepath = Path('/content/ml_avg_out.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)  
  avg_res.to_csv(filepath)

In [ ]:
pred1 = pred

In [ ]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def arrange_predictions_and_targets(pred, j): 

  #create dataframe that contain predictions and targets
  temp = pd.DataFrame()
  temp["prediction_value"] = pred["pred"][j]
  temp["target"] = pred["target"].values.tolist()[j]

  #sort in descending order respect to the predictions
  temp.sort_values(by=['prediction_value'], inplace=True, ascending=False, ignore_index=True) 

  #compute the sigmoid function over the predictions (in order to obtain values in [0, 1])
  for index, value in enumerate(temp["prediction_value"]):
    temp.loc[index, "sigmoid_value"] = sigmoid(value)

  #convert predictions from real value to 0 or 1
  temp.loc[temp["sigmoid_value"] >= 0.5, "prediction"] = 1
  temp.loc[temp["sigmoid_value"] < 0.5, "prediction"] = 0
  temp.drop(['prediction_value', "sigmoid_value"], inplace=True, axis=1)

  #cast dataframe type to int
  temp = temp.astype(int)

  return temp

def getTpTnFpFn(df): 

  TP = len(df.loc[df[["target", "prediction"]].eq(1).all(1)])
  TN = len(df.loc[df[["target", "prediction"]].eq(0).all(1)])
  FP = len(df.loc[(df["target"] == 0) & (df["prediction"] == 1)]) 
  FN = df.shape[0] - (TP + TN + FP)

  return TP, TN, FP, FN

def get_rank_at_k(df, total_relevant_docs, K=95): 
  
  relevant_docs = 0
  
  for j in range(len(df)): 
    
    if df.loc[j, "target"] == 1: 
      relevant_docs = relevant_docs + 1
    
    recall_k = relevant_docs / total_relevant_docs
    
    if recall_k >= K/100: 
      return j

  print("Error")
  return 0

def update_results(pred_df, df, model, K=95): 

  #compute TP, TN, FP, FN
  TP, TN, FP, FN = getTpTnFpFn(pred_df)
  retrieved_docs = TP + FP
  relevant_docs = TP + FN
  not_relevant_docs = TN + FP

  #EVALUATION MEASURES AT 95% RECALL

  #selection of the rows where recall@95
  last_positive_rank = get_rank_at_k(pred_df, relevant_docs) 
  pred_df_95 = pred_df[:last_positive_rank+1]
  pred_df_5 = pred_df[last_positive_rank+1:]
  TP_95, TN_95, FP_95, FN_95 = getTpTnFpFn(pred_df_95)
  TP_5, TN_5, FP_5, FN_5 = getTpTnFpFn(pred_df_5)

  #TRUE NEGATIVE RATE AT 95% RECALL
  if not_relevant_docs==0: 
    print("Number of not relevant docs equals to 0")
    TNR_k = 0
  else:
    TNR_k = TN_5/not_relevant_docs

  #PRECISION@95
  if retrieved_docs==0: 
    print("Number of retrieved docs equals to 0")
    precision_k = 0
  else:
    precision_k = TP_95/retrieved_docs 

  #WSS@95
  N = len(pred_df)
  WSS_k = ((N - last_positive_rank)/N) - ((100-K)/100)

  #update results dataset
  condition = (avg_res["df"] == df) & (avg_res["model"] == model)
  avg_res.loc[condition, "true_negative_rate@95"] = round(TNR_k*100, 3)
  avg_res.loc[condition, "precision@95"] = round(precision_k*100, 3)
  avg_res.loc[condition, "wss@95"] = round(WSS_k*100, 3)

  print("N:", N)
  print("Last positive rank:", last_positive_rank)
  print()

In [ ]:
for j in range(len(pred)): 

  data = arrange_predictions_and_targets(pred, j)

  update_results(data, df=pred.loc[j, "df"], model=pred.loc[j, "model"])

In [ ]:
avg_res

In [ ]:
with open(path + "/ml_out_2.csv", 'w', encoding = 'utf-8-sig') as f:
  avg_res.to_csv(f)

In [ ]:
y